In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium
import requests
from pandas.io.json import json_normalize

# Interface with GIS

In [2]:
url = 'https://gis.atlantaga.gov/dpcd/rest/services/OpenDataService/FeatureServer/4/query?where=1%3D1&outFields=LOCALID,NAME,NPU,GLOBALID&outSR=4326&f=json'

# Interface with Foursquare
https://developer.foursquare.com/docs/api/venues/trending

In [3]:
# @hidden_cell
client_secret = 'PLV5C4EDNBSRH4AMBKOXNGHFZWHEWCGULGTQLPQZVQFJFYZO'
client_id = 'HE3V0C15YMFATHFBWRAZ0B32H12HYYSJ1HJ523U02CKPEO3M'
ver = '20180604'

In [4]:
lat = 33.755845
long = -84.38901
limit = 500
radius=500

In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&limit={}&radius={}'.format(client_id, client_secret, lat, long, ver, limit,radius)

In [6]:
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
dataframe = json_normalize(items)

In [7]:
dataframe.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.name,venue.delivery.url,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-57d42371498ebd10aa19e10b-0,"[{'id': '5bae9231bedf3950379f89d4', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 33.75572717691757...",33.755727,-84.389400,"Downtown Atlanta, Atlanta, GA",30303,GA,Fish Bowl Poké,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d920f119d0f721e6b692473-1,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 33.75549602545936...",33.755496,-84.389006,NaN,30303,GA,Blossom Tree,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a92bba3f964a5207e1d20e3-2,"[{'id': '4bf58dd8d48988d14a941735', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 33.75560963018946...",33.755610,-84.389530,NaN,30303,GA,Dua Vietnamese Noodle Soup,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a89913bf964a5206d0820e3-3,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",326067,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/reubens-del...,...,"[{'label': 'display', 'lat': 33.75568700591916...",33.755687,-84.389442,NaN,30303,GA,Reuben's Deli,0,[],425632457
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b6315eaf964a520e8612ae3-4,"[{'id': '4bf58dd8d48988d119941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,...,"[{'label': 'display', 'lat': 33.75511963865315...",33.755120,-84.389413,NaN,30303,GA,Anatolia Cafe & Hookah Lounge,0,[],37008017


In [8]:
dataframe.shape

(55, 28)

In [9]:
dataframe.columns.values

array(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.delivery.id', 'venue.delivery.provider.icon.name',
       'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes',
       'venue.delivery.provider.name', 'venue.delivery.url', 'venue.id',
       'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'], dtype=object)

In [10]:
data=pd.DataFrame()
data['name'] = dataframe['venue.name']
data['categories'] = dataframe['venue.categories']
data['latitude'] = dataframe['venue.location.lat']
data['longitude'] = dataframe['venue.location.lng']
data['neighborhood'] = dataframe['venue.location.neighborhood']
data.neighborhood.unique()


array(['Downtown Atlanta, Atlanta, GA', nan, 'Downtown Atlanta',
       'Downtown'], dtype=object)

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
data['categories'] = data.apply(get_category_type, axis=1)

In [13]:
data.head()

,name,categories,latitude,longitude,neighborhood
0,Fish Bowl Poké,Poke Place,33.755727,-84.389400,"Downtown Atlanta, Atlanta, GA"
1,Blossom Tree,Korean Restaurant,33.755496,-84.389006,NaN
2,Dua Vietnamese Noodle Soup,Vietnamese Restaurant,33.755610,-84.389530,NaN
3,Reuben's Deli,Deli / Bodega,33.755687,-84.389442,NaN
4,Anatolia Cafe & Hookah Lounge,Hookah Bar,33.755120,-84.389413,NaN


### Visualize results

In [20]:
cat_list = data.categories.unique()
colors = plt.cm.Dark2.colors
print(colors)
    

((0.10588235294117647, 0.6196078431372549, 0.4666666666666667), (0.8509803921568627, 0.37254901960784315, 0.00784313725490196), (0.4588235294117647, 0.4392156862745098, 0.7019607843137254), (0.9058823529411765, 0.1607843137254902, 0.5411764705882353), (0.4, 0.6509803921568628, 0.11764705882352941), (0.9019607843137255, 0.6705882352941176, 0.00784313725490196), (0.6509803921568628, 0.4627450980392157, 0.11372549019607843), (0.4, 0.4, 0.4))


In [23]:
venues_map = folium.Map(location=[lat, long], zoom_start=15)

for index, row in data.iterrows():
    lat = row['latitude']
    long = row['longitude']
    label = row['name']
    category = row['categories']
    
    folium.Circle(
        [lat, long],
        popup=label,
        radius=10,
        fill=True
        ).add_to(venues_map)

# display map
venues_map

# K Nearest Neighbors Classifer

In [24]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
X = data['latitude', 'longitude', 'categories']
y = data['neighborhood']
clf = NearestCentriod()
clf.fit(X, y)


KeyError: ('latitude', 'longitude', 'categories')